In [5]:
import FinanceDataReader as fdr
import pandas as pd
import mplfinance as mpf
import numpy as np
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick2_ohlc

In [6]:
df_kospi = fdr.StockListing('KOSPI')
kospi_code_list = []
for index, row in df_kospi.iterrows():
    kospi_code_list.append(row['Code'])
print(len(kospi_code_list))

951


In [7]:
# 삼성전자 주가 확인
df = fdr.DataReader('005930', '2023-01-01', '2023-09-19')
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2023-01-02,55500,56100,55200,55500,10031448,0.003617
2023-01-03,55400,56000,54500,55400,13547030,-0.001802
2023-01-04,55700,58000,55600,57800,20188071,0.043321
2023-01-05,58200,58800,57600,58200,15682826,0.006920
2023-01-06,58300,59400,57900,59000,17334989,0.013746
...,...,...,...,...,...,...
2023-09-13,71100,71600,70300,70900,15955797,0.005674
2023-09-14,71200,71800,70800,71700,21041407,0.011283
2023-09-15,71700,72300,71400,72000,17823512,0.004184


In [8]:
# 삼성전자 주가로 데이터프레임 출력 형식 확인
for index, value in enumerate(df.index[:-1]):
    today = df.iloc[index, 3]
    tomorrow = df.iloc[index+1, 3]
    
    if today > tomorrow:
        label = 0
    else:
        label = 1
    df.loc[value, 'label'] = label

df.head(10)

,Open,High,Low,Close,Volume,Change,label
Date,,,,,,,
2023-01-02,55500,56100,55200,55500,10031448,0.003617,0.0
2023-01-03,55400,56000,54500,55400,13547030,-0.001802,1.0
2023-01-04,55700,58000,55600,57800,20188071,0.043321,1.0
2023-01-05,58200,58800,57600,58200,15682826,0.006920,1.0
2023-01-06,58300,59400,57900,59000,17334989,0.013746,1.0
2023-01-09,59700,60700,59600,60700,18640107,0.028814,0.0
2023-01-10,60200,61100,59900,60400,14859797,-0.004942,1.0
2023-01-11,61000,61200,60300,60500,12310751,0.001656,1.0
2023-01-12,61100,61200,59900,60500,16102561,0.000000,1.0


In [9]:
import datetime

today = datetime.date.today()
yesterday = (today - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

for code in kospi_code_list:
    df = fdr.DataReader(code, '2023-08-01', yesterday)
    
    for index, value in enumerate(df.index[:-1]):
        today = df.iloc[index, 3]
        tomorrow = df.iloc[index+1, 3]
        
        # 다음날 하락/상승 라벨링
        if today > tomorrow:
            label = 0
        else:
            label = 1
        df.loc[value, 'label'] = label
        
    for i in range (0, df.shape[0] - 10):
        
        data = df.iloc[i:i+10, :]
        
        # 10일간의 데이터 다음 날 가격이 어떻게 변했는지?
        label = data.iloc[-1]['label'] 
        
        fig = plt.figure(figsize=(1,1))
        ax = fig.add_subplot(1, 1, 1)
        candlestick2_ohlc(ax, data['Open'], data['High'],
                          data['Low'], data['Close'], width=0.7, colorup='r', colordown='b')
        ax.grid(False)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax.axis('off')
        plt.tight_layout()
        plt.draw()
        # 주가 상승, 하락 캔들차트 디렉토리 구분하여 저장
        if label == 0: 
            plt.savefig('C:/ssafy/candle/0/{}_{}_{}.png'.format(code, i, label), dpi=fig.dpi)
        else:
            plt.savefig('C:/ssafy/candle/1/{}_{}_{}.png'.format(code, i, label), dpi=fig.dpi)
        plt.close()